In [1]:
from device import Device
from rgf import GreensFunction
import scipy as sp
from helper import Helper_functions
import scipy.sparse as spa
import numpy as np
import scipy.sparse as sp
from lead_self_energy import LeadSelfEnergy
from scipy.sparse import bmat, identity, random
from scipy.sparse.linalg import eigsh, eigs
import time

In [2]:
device = Device()
rgf = GreensFunction(device_state=device)
lead = LeadSelfEnergy(device)
time_start = time.time()
rgf.sparse_rgf_G_R(0.1, 0.1)
time_end = time.time()

print(time_end - time_start)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
20.28970766067505


In [3]:
time_start = time.time()
dagger = lambda A: np.conjugate(A.T)
side = "left"
Hpp_matrices = [None] * lead.P
HpP_matrices = [None] * lead.P
hPP,hPP1 = lead.get_layer_hamiltonian(lead.P, side)

HPP = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - hPP))
Hpp_matrices[-1], HpP_matrices[-1] = HPP, HPP
for i in range(lead.P - 1, 0, -1):
    
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    Hpp = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - \
        hpp - hpp1 @ Hpp_matrices[i] @ dagger(hpp1)))
    Hpp_matrices[i - 1] = Hpp
    HpP = Hpp_matrices[i - 1] @ hpp1 @ HpP_matrices[i]
    HpP_matrices[i - 1] = HpP
    
C22 = Hpp_matrices[1]
C2P = HpP_matrices[1]
C_matrices = [None] * 4
C_matrices[1] = C22
for p in range(3, lead.P + 1):
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    C_matrices[p - 1] = Hpp_matrices[p - 1] + Hpp_matrices[p -1] @ (hpp1 @ C_matrices[p -2] @ dagger(hpp1)) @ Hpp_matrices[p - 1] 

h11, h12 = lead.get_layer_hamiltonian(1, side)

XIs = h11 + h12 @ C_matrices[1] @ dagger(h12)
XI = XIs + dagger(hPP1) @ C_matrices[-1] @ hPP1
PI = h12 @ C2P @ hPP1





In [ ]:
XIs = spa.csc_matrix(XIs)
XI = spa.csc_matrix(XI)
PI = spa.csc_matrix(PI)
I = np.eye(XI.shape[0], dtype=XI)
Z = I * 0
D = lead.E * I.copy() - XI
T = -PI

A = bmat([
    [Z, I],
    [-T.conj().T, -D]
], format='csc')

B = bmat([
    [I, Z],
    [Z, T]
], format='csc')

eigenvalues, eigenvectors = eigs(A, M=B, sigma=1.0, which='LM')




def construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, n_dim, epsilon=0.1):
    abs_vals = np.abs(eigenvalues)
    

    is_propagating = np.isclose(abs_vals, 1.0)
    is_evanescent = (abs_vals < 1.0) & (abs_vals > epsilon)
    
    selected_indices = np.where(is_propagating | is_evanescent)[0]
    
    if len(selected_indices) == 0:
        return np.array([], dtype=complex), np.array([],dtype=complex)
        
    filtered_eigenvalues = eigenvalues[selected_indices]
    filtered_eigenvectors = eigenvectors[:, selected_indices]

    Lambda_plus = np.diag(filtered_eigenvalues)
    U_plus = filtered_eigenvectors[:n_dim, :]

    return U_plus, Lambda_plus

U_plus, Lambda = construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, T.shape[0], epsilon=0.1)

U_pseudo = np.linalg.pinv(U_plus)
F = U_plus @ Lambda @ U_pseudo

Y = np.linalg.solve(lead.E * I - XIs.toarray() - PI.toarray() @ F, dagger(h12.toarray()))
self_energy = h12 @ Y
time_end = time.time()
print(f"time is: {time_end - time_start}")

ValueError: blocks must be 2-D

In [5]:
print(self_energy)

[[ 1.69279096e+00+1.22883006e+00j  9.96235960e-02-1.28217667e-01j
   5.70608764e-02+1.60927263e-01j ...  1.74657776e-03+2.31092555e-02j
   2.87087755e-02-1.16773312e-02j -3.95688275e-03+4.67418354e-03j]
 [ 4.47584580e-01+3.88992756e-01j  1.33837514e+01+1.27024888e-02j
  -1.74381131e+01-2.31756296e-02j ... -2.81235122e-02-1.26466163e-02j
  -1.26515986e-02+1.53568549e-03j  1.37876813e-02+1.00908696e-02j]
 [ 8.30936090e-01+1.04280846e-01j -1.73351346e+01+1.69901122e-02j
   1.33287921e+01-3.00258496e-02j ... -4.47370889e-02-9.86581871e-03j
  -1.38625910e-02+1.32043306e-02j  1.63736453e-02+1.03327875e-02j]
 ...
 [-8.56230669e-01-2.48882160e-01j -2.00423706e-01-1.44123287e-01j
  -6.17729871e-02+2.83799715e-02j ... -1.36947433e+01+3.66389831e-02j
   2.87409982e-02-3.97328909e-02j -2.37220188e-02-9.47157179e-03j]
 [-4.54453260e-02-1.94195604e-01j  3.84628622e-02-7.92212115e-02j
   9.04290507e-03-2.65205919e-02j ... -8.39383295e-03+5.13599186e-02j
  -1.37598154e+01+8.13976938e-03j  2.89339042e-

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)

[Atom(0, 0, 0), Atom(0, 0.5, 0.5)]
[Atom(0, 0, 1), Atom(0, 0.5, 1.5)]
[Atom(0, 0, 2), Atom(0, 0.5, 2.5)]
[Atom(0, 0, 3), Atom(0, 0.5, 3.5)]
[Atom(0, 0, 4), Atom(0, 0.5, 4.5)]
[Atom(0, 0, 5), Atom(0, 0.5, 5.5)]
[Atom(0, 0, 6), Atom(0, 0.5, 6.5)]
[Atom(0.25, 0.25, 0.25), Atom(0.25, 0.75, 0.75)]
[Atom(0.25, 0.25, 1.25), Atom(0.25, 0.75, 1.75)]
[Atom(0.25, 0.25, 2.25), Atom(0.25, 0.75, 2.75)]
[Atom(0.25, 0.25, 3.25), Atom(0.25, 0.75, 3.75)]
[Atom(0.25, 0.25, 4.25), Atom(0.25, 0.75, 4.75)]
[Atom(0.25, 0.25, 5.25), Atom(0.25, 0.75, 5.75)]
[Atom(0.25, 0.25, 6.25), Atom(0.25, 0.75, 6.75)]
2
<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 5152 stored elements and shape (280, 280)>
  Coords	Values
  (0, 0)	(7.8483199999999975+0j)
  (0, 1)	(-6.661338147750939e-16+0j)
  (0, 2)	(1.1102230246251565e-15+0j)
  (0, 3)	(-9.999999999999996+0j)
  (0, 140)	(-1.9352073963782712-0.30650674038727554j)
  (0, 141)	(1.7253359913073014+0.27326637535498594j)
  (0, 142)	(1.7253359913073014+0.273266

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)

[[ 7.84832000e+00+0.j -6.66133815e-16+0.j  1.11022302e-15+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 8.88178420e-16+0.j  1.42292500e+01+0.j -1.00000000e+01+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.00000000e+01+0.j  1.42292500e+01+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   1.37895000e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  1.37895000e+01+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  1.91165000e+01+0.j]]


In [ ]:
def surface_gf(Energy, H00, H10, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """

    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0
    first_time = True

    while err > tol:
        if first_time:
            inv_E = Helper_functions.sparse_inverse(spa.csr_matrix(Energy * I) - spa.csr_matrix(epsilon))
            first_time = False
        else:

            inv_E = np.linalg.solve(Energy * I - epsilon, I)
    
        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)


surface_gf(0, H00, H10, tol=1e-3)

start
[[-0.46265481-3.43058915e-15j  0.26020359+1.07328341e-01j
   0.22438539+1.30259377e-01j ... -0.06810719-6.98355909e-03j
   0.0692144 -3.15319604e-02j -0.14372304-2.32399663e-02j]
 [ 0.26020359-1.07328341e-01j -0.23397533-5.30563211e-16j
  -0.18724291-1.82886728e-02j ...  0.02738157-1.92212765e-02j
  -0.01557864+3.36768101e-02j  0.06061459-3.72506451e-02j]
 [ 0.22438539-1.30259377e-01j -0.18724291+1.82886728e-02j
  -0.23001246-7.12869148e-16j ...  0.02137388-2.46542704e-02j
  -0.00679168+3.54891358e-02j  0.04851731-4.95490009e-02j]
 ...
 [-0.06810719+6.98355909e-03j  0.02738157+1.92212765e-02j
   0.02137388+2.46542704e-02j ... -0.09098765-1.23251985e-16j
   0.01310373-6.53896350e-03j -0.03164499-2.48658897e-03j]
 [ 0.0692144 +3.15319604e-02j -0.01557864-3.36768101e-02j
  -0.00679168-3.54891358e-02j ...  0.01310373+6.53896350e-03j
  -0.09913441-1.44218598e-16j  0.02828178+1.64868977e-02j]
 [-0.14372304+2.32399663e-02j  0.06061459+3.72506451e-02j
   0.04851731+4.95490009e-02j ... -0

array([[-0.19135261+7.48661586e-02j,  0.2118514 -9.00054160e-02j,
         0.21315018-8.91832769e-02j, ..., -0.00394714-6.10309501e-03j,
         0.00682916+6.33059985e-03j, -0.00664039-1.52196826e-02j],
       [ 0.08016425+1.05039217e-01j, -0.08153963-1.07665448e-01j,
        -0.03719187-1.07388794e-01j, ...,  0.00311232+1.45570970e-03j,
        -0.00432909-8.31659925e-06j,  0.00699558+3.99447844e-03j],
       [ 0.04821244+1.38463126e-01j, -0.0066478 -1.39331436e-01j,
        -0.04586694-1.39069810e-01j, ...,  0.00346526+5.96738055e-04j,
        -0.00390684+1.34903520e-03j,  0.00737847+2.26683188e-03j],
       ...,
       [ 0.02583285+1.58618095e-02j, -0.03115715-1.38741753e-02j,
        -0.03105831-1.48418945e-02j, ..., -0.07339599+1.93945977e-03j,
         0.00216878+5.16391693e-03j, -0.00123908-1.48397039e-03j],
       [ 0.03396117-3.76533231e-02j, -0.04233916+3.50477663e-02j,
        -0.04305065+3.51312218e-02j, ..., -0.00134742+1.44865477e-03j,
        -0.08046237-2.50662427e-03j

In [ ]:
def surface_gf(Energy, H00 : np.ndarray, H10: np.ndarray, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """
    
    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0

    while err > tol:
        inv_E = np.linalg.solve(Energy * I - epsilon, I)

        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)

surface_gf(0, H00, H10)

array([[-0.19135261+7.48661586e-02j,  0.2118514 -9.00054160e-02j,
         0.21315018-8.91832769e-02j, ..., -0.00394714-6.10309501e-03j,
         0.00682916+6.33059985e-03j, -0.00664039-1.52196826e-02j],
       [ 0.08016425+1.05039217e-01j, -0.08153963-1.07665448e-01j,
        -0.03719187-1.07388794e-01j, ...,  0.00311232+1.45570970e-03j,
        -0.00432909-8.31659933e-06j,  0.00699558+3.99447844e-03j],
       [ 0.04821244+1.38463126e-01j, -0.0066478 -1.39331436e-01j,
        -0.04586694-1.39069810e-01j, ...,  0.00346526+5.96738055e-04j,
        -0.00390684+1.34903520e-03j,  0.00737847+2.26683188e-03j],
       ...,
       [ 0.02583285+1.58618095e-02j, -0.03115715-1.38741753e-02j,
        -0.03105831-1.48418945e-02j, ..., -0.07339599+1.93945977e-03j,
         0.00216878+5.16391693e-03j, -0.00123908-1.48397039e-03j],
       [ 0.03396117-3.76533231e-02j, -0.04233916+3.50477663e-02j,
        -0.04305065+3.51312218e-02j, ..., -0.00134742+1.44865477e-03j,
        -0.08046237-2.50662427e-03j

In [ ]:
from helper import Helper_functions

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)
Helper_functions.sparse_inverse(-H00)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)
Helper_functions.sparse_inverse(-H00)

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 15164 stored elements and shape (560, 560)>
  Coords	Values
  (0, 0)	(7.8483199999999975+0j)
  (0, 1)	(-6.661338147750939e-16+0j)
  (0, 2)	(1.1102230246251565e-15+0j)
  (0, 3)	(-9.999999999999996+0j)
  (0, 10)	(-1.9352073963782712-0.30650674038727554j)
  (0, 11)	(1.7253359913073014+0.27326637535498594j)
  (0, 12)	(1.7253359913073014+0.27326637535498594j)
  (0, 13)	(1.7253359913073014+0.27326637535498594j)
  (0, 14)	(-1.3029177291723641-0.20636189532388063j)
  (0, 15)	(-1.3029177291723641-0.20636189532388063j)
  (0, 16)	(-1.3029177291723641-0.20636189532388063j)
  (0, 18)	(-2.5054621766260616e-16-3.9682622459921325e-17j)
  (0, 19)	(-1.503557960887978-0.23814018613074345j)
  (1, 0)	(-6.661338147750939e-16+0j)
  (1, 1)	(14.229249999999999+0j)
  (1, 2)	(-9.999999999999998+0j)
  (1, 3)	(-8.881784197001252e-16+0j)
  (1, 10)	(-1.7253359913073014-0.27326637535498594j)
  (1, 11)	(0.3514919420620609+0.055670854522083854j)
  (1, 12)

array([[-0.0090192 -5.10008702e-17j,  0.10186053+2.42719853e-03j,
         0.09749155+2.67674066e-03j, ...,  0.0014451 -4.54025400e-04j,
        -0.00222137-1.92015229e-03j,  0.00086335+2.20492192e-03j],
       [ 0.10186053-2.42719853e-03j, -0.10101744+3.55248541e-18j,
        -0.06147344+9.38219590e-05j, ...,  0.00029316-1.49862273e-04j,
        -0.00054636-3.29359108e-04j,  0.00026605+4.40402709e-04j],
       [ 0.09749155-2.67674066e-03j, -0.06147344-9.38219590e-05j,
        -0.10234001+4.73555206e-18j, ...,  0.00018871-1.26928282e-04j,
        -0.00040153-1.80788273e-04j,  0.00021862+2.80061504e-04j],
       ...,
       [ 0.0014451 +4.54025400e-04j,  0.00029316+1.49862273e-04j,
         0.00018871+1.26928282e-04j, ..., -0.06489748-4.13012478e-18j,
        -0.00102763-3.67162632e-04j,  0.0004483 -9.76825058e-05j],
       [-0.00222137+1.92015229e-03j, -0.00054636+3.29359108e-04j,
        -0.00040153+1.80788273e-04j, ..., -0.00102763+3.67162632e-04j,
        -0.07130404-8.87536834e-19j

In [ ]:
H = device.hamiltonian.create_sparse_hamlitonian(0.1)